In [1]:
!wget -nc --no-check-certificate https://www.crcv.ucf.edu/data/UCF50.rar
!unrar x UCF50.rar -inul -y

--2021-10-02 14:06:13--  https://www.crcv.ucf.edu/data/UCF50.rar
Resolving www.crcv.ucf.edu (www.crcv.ucf.edu)... 132.170.214.127
Connecting to www.crcv.ucf.edu (www.crcv.ucf.edu)|132.170.214.127|:443... connected.
  Unable to locally verify the issuer's authority.
HTTP request sent, awaiting response... 200 OK
Length: 3233554570 (3.0G) [application/rar]
Saving to: ‘UCF50.rar’

UCF50.rar           100%[===================>]   3.01G  26.7MB/s    in 1m 56s  

2021-10-02 14:08:09 (26.7 MB/s) - ‘UCF50.rar’ saved [3233554570/3233554570]



In [3]:
#!unrar e UCF101.rar
!rm *.rar

In [4]:
%cd /content/UCF50/data
!ls

[Errno 2] No such file or directory: '/content/UCF50/data'
/content
sample_data  UCF50


In [5]:
image_height, image_width = 64, 64
max_images_per_class = 2000
seq_len = 70

dataset_directory = "/content/UCF50"
classes_list = ["Biking", "JumpingJack", "Kayaking", "PlayingGuitar", "Rowing", "Skijet", "TaiChi"]

model_output_size = len(classes_list)

In [7]:
def frames_extraction(video_path):
    # Empty List declared to store video frames
    frames_list = []
    
    # Reading the Video File Using the VideoCapture
    video_reader = cv2.VideoCapture(video_path)
    count = 1
 
    while count <= seq_len:
    # Iterating through Video Frames
      while True:

          # Reading a frame from the video file 
          success, frame = video_reader.read() 

          # If Video frame was not successfully read then break the loop
          if not success:
              break

          # Resize the Frame to fixed Dimensions
          resized_frame = cv2.resize(frame, (image_height, image_width))
          
          # Normalize the resized frame by dividing it with 255 so that each pixel value then lies between 0 and 1
          normalized_frame = resized_frame / 255
          
          # Appending the normalized frame into the frames list
          frames_list.append(normalized_frame)
    
    # Closing the VideoCapture object and releasing all resources. 
    video_reader.release()

    # returning the frames list 
    return frames_list

In [ ]:
def create_dataset():

    # Declaring Empty Lists to store the features and labels values.
    temp_features = [] 
    features = []
    labels = []
    
    # Iterating through all the classes mentioned in the classes list
    for class_index, class_name in enumerate(classes_list):
        print(f'Extracting Data of Class: {class_name}')
        
        # Getting the list of video files present in the specific class name directory
        files_list = os.listdir(os.path.join(dataset_directory, class_name))

        # Iterating through all the files present in the files list
        for file_name in files_list:

            # Construct the complete video path
            video_file_path = os.path.join(dataset_directory, class_name, file_name)

            # Calling the frame_extraction method for every video file path
            frames = frames_extraction(video_file_path)
            if len(frames) == seq_len:

            # Appending the frames to a temporary list.
              temp_features.extend(frames)

              features.append(temp_features)

              y = [0]*len(classes_list)
              y[classes_list.index(c)] = 1
        
        # Adding randomly selected frames to the features list
        #features.extend(random.sample(temp_features, max_images_per_class))

        # Adding Fixed number of labels to the labels list
              #labels.extend([class_index] * max_images_per_class)
        
        # Emptying the temp_features list so it can be reused to store all frames of the next class.
        temp_features.clear()

    # Converting the features and labels lists to numpy arrays
    features = np.asarray(features)
    labels = np.array(labels)  

    return features, labels

In [12]:
from tensorflow.python.keras.metrics import Metric